<a href="https://colab.research.google.com/github/KwonDoRyoung/AdvancedBasicEducationProgram/blob/main/abep09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인공지능
- 인공지능의 학습 및 활용을 위한 코딩은 3 단계로 이루어진다.
  1. 데이터 수집 및 전처리
    - Task에 맞는 데이터를 수집
    - 학습에 알맞은 형태로 데이터를 검수 및 분할(Train & Validation)
      - Validation 은 Train 과 겹치면 안됨.
    - Normalize, 데이터 강화, 데이터 분석, 불필요한 정보 삭제 등
    - Open dataset 활용을 권장
      - 딥러닝은 대용량(빅) 데이터기반에서 매우 효과적임
      - 오픈 데이터셋으로 학습 후, 적은 데이터(수집한 데이터 등)로 Fine-tuning을 하는 것이 일반화 성능에서 좋음 
    ---
  2. 인공지능 모델 학습
    - 모델 생성(nn.Module) 및 선언
    - 학습 및 평가를 위한 데이터 클래스 생성(Dataset)
      1. Pytorch에서 만들어둔 함수 또는 클래스 활용
      2. 수집된 데이터를 위한 Custom 데이터셋 클래스 생성
    - 학습을 위한 mini-batch 함수 호출(DataLoader)
    - 손실 함수(Objective Function or Criterion) 선언
      - 주어진 Task에 따라서 달라짐
      - 분류에서는 대표적으로 Cross Entropy Loss
      - 회귀에서는 대표적으로 Mean Square Error
    - 최적화 기법(Optimization) 선언(torch.optim)
      - 주어진 손실함수의 최소지점을 찾기 위한 기법
      - 대표적으로 Gradient Descent 방식이 있음
    - 반복문 for를 활용한 반복 학습 및 Overfitting 방지를 위한 자체 평가 단계
      - 데이터를 한번만 봐서는 학습되지 않음 > 반복학습
      - 반복학습을 하면 모델이 학습데이터에 의존적으로 변할 수 있음. 특히 적은 데이터에서 두드러짐.
        - 일반화 성능과 학습의 정도를 파악하기 위해 Validation 데이터로 평가를 진행
    ---
  3. 인공지능 모델 평가
    - 테스트 데이터를 통한 평가 단계 

## 1. 모델 생성(nn.Module) 및 선언 - 분류(Classification) 인공지능 신경망 선언
    
- nn.Module 은 Pytorch 에서 제공하는 인공신경망 클래스
- 직접 Backpropagation 연산을 선언할 필요없음
- Python의 상속을 이용
- def __init__(self, *):
  - 만들고자 하는 신경망의 구조를 선언하는 부분
  - nn에 있는 클래스를 활용하여 구조를 만들어야됨
- def forward(self, inputs, *):
  - __init__ 에서 선언한 구조들의 흐름을 만드는 곳
  - 신경망에 입력(inputs)을 넣어서 출력을 도출할 수 있음(함수의 return 인자를 활용)
- 다른 함수를 클래스 밑에 정의할 수 있음
-----
기본구조는
```python 
class <신경망의 이름>(nn.Module):
  def __init__(self, 매개변수1, 매개변수2, ...):
    # 신경망 구조 선언
    self.layer1 = nn.Linear(...)
    self.layer2 = nn....
    ...
    self.layer_n = nn....

  def forward(self, inputs, 매개변수1, ...):
    # 위에서 선언한 신경망 호출
    x = self.layer(inputs)
    x = self.layer2(x)
    ...
    x = self.layer_n(x)
    return x
```

### Multi-layer perceptron
- Pytroch 에서는 nn.Linear와 활성화 함수를 조합하여 만들 수 있음

  ![nn_p](./markdown_images/nn_p_t.png)
- 활성화 함수는 신경망에 있어서 중요함.
  - 선형 조합을 비선형 조합으로 변경시켜 복잡한 문제에도 일반화가 가능함
  - Sigmoid, ReLU, PReLU, GLU 등 다양함
  - https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity

In [1]:
import torch.nn as nn

class MLP(nn.Module): 
  def __init__(self,):
    super(MLP, self).__init__()
    # 1st layer 선언
    self.layer1 = nn.Linear(784, 100, bias=False)
    self.activation1 = nn.Sigmoid()
    # 2nd layer 선언
    self.layer2 = nn.Linear(100, 10, bias=False)
    self.activation2 = nn.Sigmoid()
    # 3rd layer 선언
    self.layer3 = nn.Linear(10, 10, bias=False)
  
  def forward(self, inputs):
    # 1st layer 입력
    outputs = self.layer1(inputs)
    outputs = self.activation1(outputs)
    # 2nd layer 입력
    outputs = self.layer2(outputs)
    outputs = self.activation2(outputs)
    # 3rd layer 입력
    outputs = self.layer3(outputs)
    return outputs

# 모델 선언
model2 = MLP().to("cuda")

## 2. 학습 및 평가를 위한 데이터 클래스 생성(Dataset)과 학습을 위한 mini-batch 함수 호출(DataLoader)
    

In [2]:
import torch.utils.data
import torchvision.transforms as T

from torchvision.datasets import MNIST

## MNIST: 손글씨, 아라비아 숫자에 대한, 0 ~ 9
train_dataset = MNIST(root="/content/", transform=T.Compose([T.ToTensor()]), download=True)
test_dataset = MNIST(root="/content/", transform=T.Compose([T.ToTensor()]), train=False)
# batch size 만큼 묶어주는 함수가 필요.
# train_dataset[index] 호출 -> (image data, image target)

print(f"data 1st:{train_dataset[0][0].size()}, {train_dataset[0][1]}") # 데이터를 통해 batch learning

batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # batch size 32~64,... 
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

data 1st:torch.Size([1, 28, 28]), 5


## 3. 손실 함수(Objective Function or Criterion) 선언
    

In [3]:
criterion = nn.CrossEntropyLoss()

## 4. 최적화 기법(Optimization) 선언(torch.optim)

In [4]:
lr=0.01
optimizer = torch.optim.SGD(model2.parameters(), lr=lr)

## 5. 반복문 for를 활용한 반복 학습 및 Overfitting 방지를 위한 자체 평가 단계

In [ ]:
n_epoch = 100
print("Train & Validation")
for epoch in range(n_epoch):
  model2.train() # 여기서부터는 학습 Flag
  train_loss = 0
  for step, x in enumerate(train_loader): # 한번 데이터를 다 보는 거 (batch0, batch1, batch2, ...,batch_m)
    batch_image = x[0] # torch.Size([batch_size, 1, 28, 28]): dim=4
    batch_image = torch.flatten(batch_image, start_dim=1).to("cuda")
    batch_target = x[1].to("cuda")
    outputs = model2(batch_image) # forward 함수를 활용

    loss = criterion(outputs, batch_target)
    
    train_loss += loss # 추가된 라인

    # 학습을 진행하는 곳
    optimizer.zero_grad()
    loss.backward() # 역전파 -> 미분값이 저장
    optimizer.step() # 미분값이 저장을 활용해서 최적화

    if step % 1000 == 0: 
      print(f"\t[{step}] Loss: {loss.item():.4f}")

  train_loss = train_loss/len(train_loader)
  print(f"[{epoch}/{n_epoch}] Train loss: {train_loss.item():.4f}")

  model2.eval() # 여기서부터는 평가 Flag
  valid_loss = 0
  total_accuracy = 0.0
  for i, v in enumerate(test_loader):
    batch_image = v[0] 
    batch_image = torch.flatten(batch_image, start_dim=1).to("cuda")
    batch_target = v[1].to("cuda")
    outputs = model2(batch_image) # forward 함수를 활용

    loss = criterion(outputs, batch_target)

    valid_loss += loss    
    correct = batch_target == torch.argmax(outputs, dim=1)
    total_accuracy += correct.float()

  print(len(test_loader))
  valid_loss = valid_loss/len(test_loader)
  total_accuracy = (total_accuracy/len(test_loader))*100
  print(f"Valid Loss: {valid_loss.item():.4f}, Accuracy: {total_accuracy.item():.2f}%")
  print("="*100)

Train & Validation
	[0] Loss: 2.4107
	[1000] Loss: 2.2869
[0/100] Train loss: 2.2995
10000
Valid Loss: 2.2910, Accuracy: 11.35%
	[0] Loss: 2.2733
	[1000] Loss: 2.2809
[1/100] Train loss: 2.2822
10000
Valid Loss: 2.2679, Accuracy: 14.51%
	[0] Loss: 2.2622
	[1000] Loss: 2.2323
[2/100] Train loss: 2.2338
10000
Valid Loss: 2.1715, Accuracy: 33.03%
	[0] Loss: 2.0963
	[1000] Loss: 1.9917
[3/100] Train loss: 2.0292
